##TEAM MEMBERS

Aidan Stautland, 502841; Christian Torhaug, 564355; Jens Skaug, STUDID

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
import geopandas as gpd
from geopy.distance import geodesic
from shapely.geometry import Point

ais_sample_submission.csv

In [2]:
sampledf = pd.read_csv("ais_sample_submission.csv")

sampledf["ID"] = sampledf["ID"].apply(lambda x: int(x))
sampledf["latitude_predicted"] = sampledf["latitude_predicted"].apply(lambda x: float(x))
sampledf["longitude_predicted"] = sampledf["longitude_predicted"].apply(lambda x: float(x))

ais_test.csv

In [3]:
testdf = pd.read_csv("ais_test.csv")

testdf["time"] = testdf["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
testdf["scaling_factor"] = testdf["scaling_factor"].apply(lambda x: float(x))

ais_train.csv

In [4]:
traindf = pd.read_csv("~/Ais_Data/ais_train.csv", sep="|")

def parseEta(row):
    try:
        return datetime.strptime(row['etaRaw'], "%m-%d %H:%M")
    except ValueError:
        return ""

#traindf["time"] = traindf["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
traindf["time"] = pd.to_datetime(traindf["time"]) 
traindf["cog"] = traindf["cog"].apply(lambda x: float(x) if float(x) < 360.0 else 360.0)
traindf["sog"] = traindf["sog"].apply(lambda x: float(x) if float(x) <= 30.0 else 0)
traindf["rot"] = traindf["rot"].apply(lambda x: int(x))
traindf["heading"] = traindf["heading"].apply(lambda x: int(x) if int(x) < 360.0 else 360.0)
traindf["etaRaw"] = traindf.apply(parseEta, axis=1)
traindf["latitude"] = traindf["latitude"].apply(lambda x: float(x))
traindf["longitude"] = traindf["longitude"].apply(lambda x: float(x))

traindf["navstat"] = traindf["navstat"].apply(lambda x: int(x) if x in [0, 1, 5, 8] else 15)
traindf["navstat"] = traindf["navstat"].replace(8, 0)

ports.csv

In [5]:
portsdf = pd.read_csv("ports.csv", sep="|")

portsdf["latitude"] = portsdf["latitude"].apply(lambda x: float(x))
portsdf["longitude"] = portsdf["longitude"].apply(lambda x: float(x))

schedules_to_may_2024.csv

In [6]:
schedulesdf = pd.read_csv("~/Ais_Data/schedules_to_may_2024.csv", sep="|")
schedulesdf = schedulesdf.drop_duplicates()
schedulesdf = schedulesdf.dropna(subset=['portName'])

schedulesdf["arrivalDate"] = pd.to_datetime(schedulesdf["arrivalDate"], errors='coerce')
schedulesdf["arrivalDate"] = schedulesdf["arrivalDate"].dt.tz_convert('UTC').dt.tz_localize(None)
schedulesdf["sailingDate"] = pd.to_datetime(schedulesdf["sailingDate"], errors='coerce')
schedulesdf["sailingDate"] = schedulesdf["sailingDate"].dt.tz_convert('UTC').dt.tz_localize(None)
schedulesdf["portLatitude"] = schedulesdf["portLatitude"].apply(lambda x: float(x))
schedulesdf["portLongitude"] = schedulesdf["portLongitude"].apply(lambda x: float(x))

vessels.csv

In [7]:
vehiclesdf = pd.read_csv("vessels.csv", sep="|")

vehiclesdf["CEU"] = vehiclesdf["CEU"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["DWT"] = vehiclesdf["DWT"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["GT"] = vehiclesdf["GT"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["NT"] = vehiclesdf["NT"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["vesselType"] = vehiclesdf["vesselType"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["breadth"] = vehiclesdf["breadth"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["depth"] = vehiclesdf["depth"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["draft"] = vehiclesdf["draft"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["enginePower"] = vehiclesdf["enginePower"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["freshWater"] = vehiclesdf["freshWater"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["fuel"] = vehiclesdf["fuel"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["length"] = vehiclesdf["length"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["maxHeight"] = vehiclesdf["maxHeight"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["maxSpeed"] = vehiclesdf["maxSpeed"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["maxWidth"] = vehiclesdf["maxWidth"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["rampCapacity"] = vehiclesdf["rampCapacity"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["yearBuilt"] = vehiclesdf["yearBuilt"].apply(lambda x: datetime.strptime(str(x), "%Y"))

Function for calculating time difference

In [8]:
def timeDiff(t1: datetime, t2: datetime, i="s"):
    diff = t1 - t2 if t1 >= t2  else t2 - t1
    if i == "s":
        return diff.total_seconds()
    elif i == "m":
        return diff.total_seconds() // 60
    elif i == "h":
        return diff.total_seconds() // 3600
    elif i == "d":
        return diff.days
    else:
        return 0

Adding WORKING_SIZE previous nodes to the ais_train dataframe

In [18]:
def prev_nodes_train(traindf: pd.DataFrame, WINDOW_SIZE=1):
    for j in range(1, WINDOW_SIZE + 1):
        traindf[f'latitude_{j}'] = traindf.groupby('vesselId')['latitude'].shift(j)
        traindf[f'longitude_{j}'] = traindf.groupby('vesselId')['longitude'].shift(j)
        
        traindf[f'time_{j}'] = traindf.groupby('vesselId')['time'].shift(j)
        traindf[f'time_{j}'] = (traindf['time'] - traindf[f'time_{j}']).dt.total_seconds()
    
    traindf['last_COG'] = traindf.groupby('vesselId')['cog'].shift(1)
    traindf['last_SOG'] = traindf.groupby('vesselId')['sog'].shift(1)
    traindf['last_COG'] = traindf['last_COG'].fillna(360.0)
    traindf['last_SOG'] = traindf['last_SOG'].fillna(0)
    return traindf

def prev_nodes_test(traindf: pd.DataFrame, testdf: pd.DataFrame, WINDOW_SIZE=1):
    grouped_train = traindf.groupby('vesselId')
    
    for j in range(1, WINDOW_SIZE + 1):
        testdf[f'latitude_{j}'] = None
        testdf[f'longitude_{j}'] = None
        testdf[f'time_{j}'] = None

    testdf['last_COG'] = None
    testdf['last_SOG'] = None

    for vesselId, test_vessel_data in testdf.groupby('vesselId'):
        if vesselId in grouped_train.groups:
            last_train_row = grouped_train.get_group(vesselId).iloc[-1]

            last_lat = last_train_row['latitude']
            last_lon = last_train_row['longitude']
            last_cog = last_train_row['cog']
            last_sog = last_train_row['sog']
            last_time = last_train_row['time']

            for j in range(1, WINDOW_SIZE + 1):
                testdf.loc[test_vessel_data.index, f'latitude_{j}'] = last_lat
                testdf.loc[test_vessel_data.index, f'longitude_{j}'] = last_lon
                testdf.loc[test_vessel_data.index, f'time_{j}'] = (testdf.loc[test_vessel_data.index, 'time'] - last_time).dt.total_seconds()

            testdf.loc[test_vessel_data.index, 'last_COG'] = last_cog
            testdf.loc[test_vessel_data.index, 'last_SOG'] = last_sog

    return testdf

Checking if a boat is on a schedule

In [10]:
def onSchedule(schedulesdf: pd.DataFrame, vesselId: str, time: datetime):
    if schedulesdf.index.name != 'vesselId':
        schedulesdf = schedulesdf.set_index('vesselId')
        
    if vesselId in schedulesdf.index:
        vessel_schedules = schedulesdf.loc[vesselId]
    else:
        return None
    
    fitting_schedules = (vessel_schedules["sailingDate"] >= time) & (vessel_schedules["arrivalDate"] <= time)
    if fitting_schedules.any():
        matching_schedules = vessel_schedules.loc[fitting_schedules]
        if len(matching_schedules) == 1:
            return matching_schedules.iloc[0]
    return None

Finding Last Data Point

In [11]:
def lastData(testdf: pd.DataFrame, traindf: pd.DataFrame):
    vessels = testdf["vesselId"].unique().tolist()
    indexes = []

    for v in vessels:
        vessel_data = traindf[traindf['vesselId'] == v]
        indexes.append(vessel_data.sort_values(by="time").iloc[-1].name)

    return traindf.loc[indexes].sort_values(by="time")

#print(lastData(testdf, traindf).head())

Distance

In [12]:
def distance(lat1, long1, lat2, long2):
    return geodesic((lat1, long1), (lat2, long2)).meters

Checking if a vessel is near a port

In [13]:
from sklearn.neighbors import KDTree

PORT_DIST_THRESH = 3000

ports_coords = portsdf[["latitude", "longitude"]].values
kdtree = KDTree(ports_coords, leaf_size=30)

def nearPort(portsdf: pd.DataFrame, lat, long):
    rough_dist = PORT_DIST_THRESH / 100000
    indices = kdtree.query_radius([[lat, long]], r=rough_dist)
    for i in indices[0]:
        port = portsdf.iloc[i]
        if distance(lat, long ,port["latitude"],port["longitude"]) <= PORT_DIST_THRESH:
            return port
    return None

Checking if a vessel is far from land

In [14]:
MAP_LAND_PATH = "~/datasets/ne_10m_land.zip"
MAP_OCEAN_PATH = "~/datasets/ne_10m_ocean.zip"
    
land_world = gpd.read_file(MAP_LAND_PATH)
ocean_world = gpd.read_file(MAP_OCEAN_PATH)

def farFromCoast(latitude, longitude, land_world=land_world, thresh=12):
    land_world_projected = land_world.to_crs(epsg=3857)
    point = gpd.GeoSeries([Point(longitude, latitude)], crs="EPSG:4326").to_crs(epsg=3857)
    
    if land_world_projected.contains(point.iloc[0]).any():
        return False
    
    land_boundary = land_world_projected.geometry.boundary.unary_union
    distanceToCoast = point.distance(land_boundary)
    distanceNM = distanceToCoast.min() / 1852
    
    return distanceNM > thresh

Calculating average SOG

In [15]:
def avgSOG(traindf: pd.DataFrame):
    filtered_df = traindf[traindf["navstat"] == 0]
    sog_means = filtered_df.groupby("vesselId")["sog"].mean()
    return sog_means.to_dict()

Copying Test and Train Dataframe

In [16]:
trainRFR = traindf.copy()
testRFR = testdf.copy()

Splitting data for ML

In [19]:
vessels = trainRFR["vesselId"].unique().tolist()
vessel_dict = {vessel: i for i, vessel in enumerate(vessels)}

ports = portsdf["portId"].unique().tolist()
port_dict = {port: i for i, port in enumerate(ports)}

avg = avgSOG(trainRFR)

# Adding previous nodes
window_size = 1
trainRFR = prev_nodes_train(trainRFR, window_size)
testRFR = prev_nodes_test(trainRFR, testRFR, window_size)

Creating a submission

In [20]:
def submission(sampledf: pd.DataFrame, lat_pred, lon_pred, deliver=False, name="SampleTest") -> pd.DataFrame:
    sample = sampledf.copy()

    for i in range(len(lat_pred)):
        sample.loc[i,"longitude_predicted"], sample.loc[i, "latitude_predicted"] = lon_pred[i], lat_pred[i]
    
    if deliver:
        sample.to_csv(name + ".csv", index=False)
        
    return sample

#sample = submission(sampledf, testdf) 
#sample = submission(sampledf, testdf, True, "Test1")

ML Random Forest

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_result(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f'Mean Squared Error (MSE): {mse:.4f}')
    print(f'Mean Absolute Error (MAE): {mae:.4f}')
    print(f'R^2 Score: {r2:.4f}')

y_lat = trainRFR["latitude"].copy()
y_lon = trainRFR["longitude"].copy()
X = trainRFR.drop(columns=['latitude', 'longitude'])

X = X.drop(columns=["etaRaw", "portId", "cog", "sog", "rot", "heading"])
#X["time"] = X["time"].astype("int64") / 10**9
X["time"] = pd.to_numeric(X["time"], errors='coerce')

# Splitting Navstat
X = pd.get_dummies(X, columns=["navstat"], prefix="", prefix_sep="")
X.rename(columns={"0": "Under_Way", "1": "Anchored", "5": "Moored"}, inplace=True)
X = X.drop(columns=["15"])

X["avgSOG"] = X["vesselId"].apply(lambda x: avg.get(x))
X["vesselId"] = X["vesselId"].map(vessel_dict)

X_train, X_test, y_train_lat, y_test_lat = train_test_split(X, y_lat, test_size=0.2)

#print(X_train[X_train["last_COG"].isna()])  
print(X_train.isna().sum())
clf_lat = RandomForestRegressor(n_estimators=10, random_state=42)
clf_lat.fit(X_train, y_train_lat)
y_pred_lat = clf_lat.predict(X_test)

print('Random Forest Model without Feature Engineering')
evaluate_result(y_test_lat, y_pred_lat)

#print(y_test_lat.tolist()[:10])
#print(y_pred_lat[:10])

X_train, X_test, y_train_lon, y_test_lon = train_test_split(X, y_lon, test_size=0.2)

#print(X_test.head())
clf_lon = RandomForestRegressor(n_estimators=10, random_state=42)
clf_lon.fit(X_train, y_train_lon)
y_pred_lon = clf_lon.predict(X_test)

print('Random Forest Model without Feature Engineering')
evaluate_result(y_test_lon, y_pred_lon)

#print(y_test_lon.tolist()[:10])
#print(y_pred_lon[:10])

#testRFR["nearPort"] = None
testRFR["Under_Way"] = None
testRFR["Anchored"] = None
testRFR["Moored"] = None

test = testRFR.drop(columns=['ID', 'scaling_factor'])

test["avgSOG"] = test["vesselId"].apply(lambda x: avg.get(x))

#test["time"] = test["time"].astype("int64") / 10**9
test["time"] = pd.to_numeric(test["time"], errors='coerce')
test["vesselId"] = test["vesselId"].map(vessel_dict)

cols = test.columns.tolist()
time_index = cols.index('time')
vesselId_index = cols.index('vesselId')
cols[time_index], cols[vesselId_index] = cols[vesselId_index], cols[time_index]
test = test[cols]

#print(testRFR.head())
#print(test.head())

lat_pred = clf_lat.predict(test)
lon_pred = clf_lon.predict(test)

#submission(sampledf, lat_pred, lon_pred, True, "Test1")
#print(lat_pred)
#print(lon_pred)

time             0
vesselId         0
latitude_1     539
longitude_1    539
time_1         539
last_COG         0
last_SOG         0
Under_Way        0
Anchored         0
Moored           0
avgSOG         506
dtype: int64
Random Forest Model without Feature Engineering
Mean Squared Error (MSE): 1.0942
Mean Absolute Error (MAE): 0.0588
R^2 Score: 0.9979
Random Forest Model without Feature Engineering
Mean Squared Error (MSE): 9.0290
Mean Absolute Error (MAE): 0.1116
R^2 Score: 0.9981


In [19]:
submission(sampledf, lat_pred, lon_pred, True, "prediction")

,ID,longitude_predicted,latitude_predicted
0,0,-81.497904,31.146470
1,1,120.296279,14.816910
2,2,10.822399,38.355297
3,3,172.835421,-43.537873
4,4,-6.125929,48.580984
...,...,...,...
51734,51734,-43.136336,41.848029
51735,51735,-112.443457,35.409369
51736,51736,-113.569418,23.531067
51737,51737,24.438225,55.625616
